# View Campaign and Interactions

In the first notebook you successfully built and deployed a recommendation model using deep learning with Amazon Personalize.

This notebook will use the fronting infrastructure that can automatically set context to get recommendations based on the user and device where they access it from.

Below we start with just importing libraries that we need to interact with Personalize

In [ ]:
# Imports
import boto3
import json
import numpy as np
import pandas as pd
import time
import uuid
import requests


from zipfile import ZipFile
from time import gmtime, strftime

The line below will retrieve your shared variables from the first notebook.

In [ ]:
cf_client = boto3.client('cloudformation')
cloudfront_client = boto3.client('cloudfront')

In [ ]:
%store -r

In [ ]:
try:
    describe_stack_resource_response = cf_client.describe_stack_resource(
        StackName=STACK_NAME,
        LogicalResourceId='PersonalizeACCloudFrontDistribution'
    )
    cloudfront_distribution_domain_id = describe_stack_resource_response["StackResourceDetail"]["PhysicalResourceId"]
    print('CloudFront Distribution ID is:', cloudfront_distribution_domain_id)
except:
    msg = f'CloudFormation stack {STACK_NAME} was not found. Please set the STACK_NAME properly and re-run this cell'
    sys.exit(ValueError(msg))

In [ ]:
get_distribution_response = cloudfront_client.get_distribution(
    Id=cloudfront_distribution_domain_id
)
cloudfront_distribution_domain_name=get_distribution_response.get('Distribution').get('DomainName')
print(cloudfront_distribution_domain_name)

In [ ]:
%%time
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    time.sleep(15)
    get_distribution_response = cloudfront_client.get_distribution(
        Id = cloudfront_distribution_domain_id
    )
    status = get_distribution_response['Distribution']['Status']
    print("DistributionStatus: {}".format(status))
    if status == "InProgress":
        continue
    else:
        break

In [ ]:
full_cloudfront_distribution_domain_url='https://' + cloudfront_distribution_domain_name + '/'
full_cloudfront_distribution_domain_url_with_user_id=full_cloudfront_distribution_domain_url + '?custom-user-id=' + str(user_id)
print(full_cloudfront_distribution_domain_url_with_user_id)
res = requests.get(full_cloudfront_distribution_domain_url_with_user_id, timeout=10)
print(res)

##### NOTE: Invoke this URL from different device types like Smartphones, TV, Tablet, etc. and see the difference.

Also notice how the recommendations are filtered that are only allowed in viewer's country.

In [ ]:
item_list = res.json()['itemList']

print('ITEM_ID'.ljust(8, ' '), 
      'GENRE'.ljust(20, ' '), 
      'ALLOWED_COUNTRIES'.ljust(20, ' '))

for item in item_list:
    key = item['itemId']
    df2=items_data.query(f"ITEM_ID == {key}")
    print(str(df2['ITEM_ID'].values[0]).ljust(8, ' '), 
          df2['GENRE'].values[0].ljust(20, ' '), 
          df2['ALLOWED_COUNTRIES'].values[0].ljust(20, ' '))

In cell below, the cold user we identified to test the Personalize campaign endpoint is reused. Note, since the test is being executed from a Desktop, that context will be passed on automatically from your Desktop to the campaign endpoint. Feel free to grab below URL and re-try it from your phone or tablet (or smart tv) to see how the results differ.

In [ ]:
full_cloudfront_distribution_domain_url='https://' + cloudfront_distribution_domain_name + '/'
full_cloudfront_distribution_domain_url_with_cold_user_id=full_cloudfront_distribution_domain_url + '?custom-user-id=' + str(cold_user_id)
print(full_cloudfront_distribution_domain_url_with_cold_user_id)
res = requests.get(full_cloudfront_distribution_domain_url_with_cold_user_id, timeout=10)
print(res)

##### NOTE: Invoke this URL from different device types like Smartphones, TV, Tablet, etc. and see the difference.

Also notice how the recommendations are filtered that are only allowed in viewer's country.

In [ ]:
item_list = res.json()['itemList']

print('ITEM_ID'.ljust(8, ' '), 
      'GENRE'.ljust(20, ' '), 
      'ALLOWED_COUNTRIES'.ljust(20, ' '))

for item in item_list:
    key = item['itemId']
    df2=items_data.query(f"ITEM_ID == {key}")
    print(str(df2['ITEM_ID'].values[0]).ljust(8, ' '), 
          df2['GENRE'].values[0].ljust(20, ' '), 
          df2['ALLOWED_COUNTRIES'].values[0].ljust(20, ' '))